# Organic Rankine Cycle with CoolProp

In this notebook we use the CoolProp library to evaluate the thermodynamic
states of an Organic Rankine Cycle (ORC) using octamethyltrisiloxane as the
working fluid.

The steps are:
- define the condenser (`p1`) and evaporator (`p2`) pressures,
- evaluate the saturated liquid state at the condenser outlet (state 1),
- compute the isentropic pump outlet (state 2),
- evaluate the saturated vapour state at the evaporator outlet (state 3),
- compute the isentropic turbine outlet state (state 4),
- calculate specific work and thermal efficiency of the cycle.

Make sure CoolProp is installed in your Python environment. After running the
notebook, try changing the pressure levels to see how they affect cycle
performance.


In [1]:
# Insalling CoolProp
# Using conda: conda install conda-forge::coolprop
# NOTE: it might create conflicts...in case of errors use pip:
# pip install coolprop
# REMEMBRE to install it in the virtual env if you are using one

import CoolProp.CoolProp as CP

# Define the temperature and pressure for MDM
T = 300  # Temperature in Kelvin
P = 101325  # Pressure in Pascals

fluid = "Octamethyltrisiloxane"

# Example: Get the density of Octamethyltrisiloxane (if it was defined in CoolProp)
# http://coolprop.org/coolprop/HighLevelAPI.html#parameter-table
# https://coolprop.org/fluid_properties/PurePseudoPure.html#fluid-properties

try:
    density_mdm = CP.PropsSI("D", "T", T, "P", P, fluid)
    print(f"Density of {fluid} at {T}K and {P}Pa: {density_mdm:.2f} kg/m^3")
    molar_mass = CP.PropsSI("M", "T", T, "P", P, fluid)
    print(f"Molar mass of {fluid} at {T}K and {P}Pa: {molar_mass:.2f} kg/mol")
    h1 = CP.PropsSI("H", "T", T, "P", P, fluid)
    print(f"Enthalpy of {fluid} at {T}K and {P}Pa: {h1:.2f} J/kg")
    h1 = CP.PropsSI("H", "T", T, "Q", 0, fluid)
    print(f"Enthalpy of {fluid} at {T}K and saturated liquid: {h1:.2f} J/kg")
except ValueError:
    print(f"Fluid '{fluid}' not found in CoolProp database.")

Density of Octamethyltrisiloxane at 300K and 101325Pa: 813.54 kg/m^3
Molar mass of Octamethyltrisiloxane at 300K and 101325Pa: 0.24 kg/mol
Enthalpy of Octamethyltrisiloxane at 300K and 101325Pa: -244318.43 J/kg
Enthalpy of Octamethyltrisiloxane at 300K and saturated liquid: -244396.43 J/kg


# ORC

p1 = 2 * 1e4
p2 = 9 * 1e5

working_fluid = 'Octamethyltrisiloxane'

In [2]:
p1 = 2 * 1e4
p2 = 9 * 1e5

In [3]:
working_fluid = "Octamethyltrisiloxane"

In [4]:
# 1. Condenser
h1 = CP.PropsSI(
    "H", "P", p1, "Q", 0, working_fluid
)  # Enthalpy at saturated liquid state (h1)
s1 = CP.PropsSI(
    "S", "P", p1, "Q", 0, working_fluid
)  # Entropy at saturated liquid state (s1)
print(h1 / 1000, "kJ/kg")
print(s1, "J/kg K")

-106.98062035479818 kJ/kg
-267.73206384909935 J/kg K


In [5]:
# 2. Pump: Isentropic pumping
h2 = CP.PropsSI(
    "H", "P", p2, "S", s1, working_fluid
)  # Isentropic expansion, h2 at condenser pressure
print(h2 / 1000, "kJ/kg")

-105.78764514790926 kJ/kg


In [6]:
# 3. Heating: MDM is heated up
h3 = CP.PropsSI(
    "H", "P", p2, "Q", 1, working_fluid
)  # Enthalpy at saturated vapor state
s3 = CP.PropsSI("S", "P", p2, "Q", 1, working_fluid)
print(h3 / 1000, "kJ/kg")
print(s3, "J/kg K")
print(CP.PropsSI("T", "P", p2, "Q", 1, working_fluid) - 273, "°C")

332.51775937777603 kJ/kg
672.7054524904095 J/kg K
261.7112452471009 °C


In [7]:
# 4. Turbine: Isentropic expansion
h4 = CP.PropsSI(
    "H", "P", p1, "S", s3, working_fluid
)  # Isentropic pump, h4 at evaporator pressure
print(h4 / 1000, "kJ/kg")

270.7140809278401 kJ/kg


In [8]:
# Calculate thermal efficiency of the cycle
turbine_work = h3 - h4
pump_work = h2 - h1
heat_added = h3 - h2

work = turbine_work - pump_work
eff = (turbine_work - pump_work) / heat_added

print(f"work = {work:.2f} J/kg")
print(f"efficiency = {eff:.2f}")

work = 60610.70 J/kg
efficiency = 0.14
